# Exploration - Base Dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('./data/dataset.csv')

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34627 entries, 0 to 34626
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Development Name     34620 non-null  object 
 1   Borough              34627 non-null  object 
 2   Account Name         34627 non-null  object 
 3   Location             34365 non-null  object 
 4   Meter AMR            34575 non-null  object 
 5   Meter Scope          8585 non-null   object 
 6   TDS #                34620 non-null  float64
 7   EDP                  34627 non-null  int64  
 8   RC Code              34627 non-null  object 
 9   Funding Source       34551 non-null  object 
 10  AMP #                34618 non-null  object 
 11  Vendor Name          34627 non-null  object 
 12  UMIS BILL ID         34627 non-null  int64  
 13  Revenue Month        34627 non-null  object 
 14  Service Start Date   34627 non-null  object 
 15  Service End Date     34627 non-null 

Values about cost and consumption are considered object. This situation will be fixed.

*Note: basic statistics will be showed after a first stage of preprocessing.*

The dataset, as it is, **describe a single water consumption** as per [Nyc open data doc](https://data.cityofnewyork.us/Housing-Development/Water-Consumption-And-Cost-2013-Feb-2023-/66be-66yr). Before starting our **data preparation** step, let's analyze the current situation. Let's start by identifying columns that contains null values. We will start to pre-process our data from these columns.

In [4]:
nulls = dataset.isna().sum()
nulls[nulls != 0]

Development Name        7
Location              262
Meter AMR              52
Meter Scope         26042
TDS #                   7
Funding Source         76
AMP #                   9
Rate Class              3
dtype: int64

The number of null values of meter-scope is very high. The feature will likely be removed.

## Location investigation

Let's start by extracting a sub-dataframe that contains null values for location, in order to inspect them and derive some strategy to their management.

In [5]:
null_location_data = dataset[dataset['Location'].isna()]
null_location_data[nulls[nulls != 0].index].info()

<class 'pandas.core.frame.DataFrame'>
Index: 262 entries, 22329 to 34527
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Development Name  262 non-null    object 
 1   Location          0 non-null      object 
 2   Meter AMR         260 non-null    object 
 3   Meter Scope       0 non-null      object 
 4   TDS #             262 non-null    float64
 5   Funding Source    262 non-null    object 
 6   AMP #             260 non-null    object 
 7   Rate Class        260 non-null    object 
dtypes: float64(1), object(7)
memory usage: 18.4+ KB


Fortunately, there aren't null values for TDS (which is the identifier of a development). The main problem here is that a <u>building is identified by location + TDS</u> and with missing values of locations, we can't identify a specific building inside the development. 

Let's investigate further. We want to understand whether we can discard these rows.

In [6]:
tds_null_loc_count_df = null_location_data['TDS #'].value_counts()

Here we want to check if every NaN location is associated with the same TDS.

In [7]:
# We want to filter the rows that contains TDS # values equals to the ones associated null location 
condition = np.isin(dataset['TDS #'], tds_null_loc_count_df.index.to_numpy())

dataset[condition][['Location', 'TDS #']].groupby(by='TDS #').aggregate(
    Null=('Location', lambda x: x.isna().sum()),
    Not_Null=('Location', lambda x: x.notnull().sum())
)

,Null,Not_Null
TDS #,,
4.0,1,108
51.0,229,0
209.0,3,2542
212.0,9,1622
226.0,3,1369
260.0,6,3242
274.0,3,958
284.0,8,2121


As we can see: 
- Every TDS number (from the ones that have at least one null location associated) has non-null locations associated;
- The only TDS # made only by null location is #51.

## TDS investigation

Let's see this null values of TDS what represents.

In [8]:
dataset[dataset['TDS #'].isna()][
    ['Account Name', 'AMP #', 'RC Code', 'Location', 'Borough', 'EDP']
]

,Account Name,AMP #,RC Code,Location,Borough,EDP
33294,"NDF - CENTRAL MAINTENANCE SHOP, 23 ASH ST",NaN,9900622,BLD 01,NON DEVELOPMENT FACILITY,999
33295,"NDF - CENTRAL MAINTENANCE SHOP, 23 ASH ST",NaN,9900622,BLD 01,NON DEVELOPMENT FACILITY,999
33296,"NDF - CENTRAL MAINTENANCE SHOP, 23 ASH ST",NaN,9900622,BLD 01,NON DEVELOPMENT FACILITY,999
33297,"NDF - CENTRAL MAINTENANCE SHOP, 23 ASH ST",NaN,9900622,BLD 01,NON DEVELOPMENT FACILITY,999
33902,"NDF - CENTRAL MAINTENANCE SHOP, 23 ASH ST",NaN,9900622,BLD 01,NON DEVELOPMENT FACILITY,999
34216,"NDF - CENTRAL MAINTENANCE SHOP, 23 ASH ST",NaN,9900622,BLD 01,NON DEVELOPMENT FACILITY,999
34524,"NDF - CENTRAL MAINTENANCE SHOP, 23 ASH ST",NaN,9900622,BLD 01,NON DEVELOPMENT FACILITY,999


As we can see: 
- Borough is not null and is listed as non-development-facility
- RC Code and EDP are set both to the same value for each entry. The value looks like a placeholder.

A solution is to not discard these null TDS #. We'll assign them a placeholder value like EDP one. Let's just see if there are other TDS  # associated with a borough listed like NON DEVELOPMENT FACILITY.

In [9]:
dataset[dataset['Borough'] == 'NON DEVELOPMENT FACILITY']['TDS #']

33294   NaN
33295   NaN
33296   NaN
33297   NaN
33902   NaN
34216   NaN
34524   NaN
Name: TDS #, dtype: float64

We can then proceed to assign TDS # == 999 for these kind of rows, since they are identified like non development facility.

## Borough investigation


In [10]:
dataset['Borough'].value_counts()

Borough
FHA                         16302
BROOKLYN                     6334
MANHATTAN                    5661
BRONX                        3339
QUEENS                       2904
STATEN ISLAND                  80
NON DEVELOPMENT FACILITY        7
Name: count, dtype: int64

The classic NYC boroughs are present, and we understood the meaning of NON DEVELOPMENT FACILITY. But, what exactly is FHA? A really fast research indicates the meaning of this [acronym](https://en.wikipedia.org/wiki/Federal_Housing_Administration). This means that a development's lender under FHA is protected against defaults on property's owner mortgage.

## Funding source investigation

Let's see how values are distributed under this feature.

In [11]:
dataset['Funding Source'].value_counts()

Funding Source
FEDERAL               31048
MIXED FINANCE/LLC1     2915
MHOP                    436
MIXED FINANCE/LLC2      118
SECTION 8                27
NON-DEVELOPMENT           7
Name: count, dtype: int64

Let's derive a dataframe to inspect with the VS code Data Wrangler tool.

In [12]:
null_funding_source_df = dataset[dataset['Funding Source'].isna()]
null_funding_source_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76 entries, 21219 to 21294
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Development Name     76 non-null     object 
 1   Borough              76 non-null     object 
 2   Account Name         76 non-null     object 
 3   Location             76 non-null     object 
 4   Meter AMR            76 non-null     object 
 5   Meter Scope          0 non-null      object 
 6   TDS #                76 non-null     float64
 7   EDP                  76 non-null     int64  
 8   RC Code              76 non-null     object 
 9   Funding Source       0 non-null      object 
 10  AMP #                76 non-null     object 
 11  Vendor Name          76 non-null     object 
 12  UMIS BILL ID         76 non-null     int64  
 13  Revenue Month        76 non-null     object 
 14  Service Start Date   76 non-null     object 
 15  Service End Date     76 non-null     obj

In [13]:
null_funding_source_df[['Borough', 'TDS #', 'Location']].value_counts()

Borough   TDS #  Location
BROOKLYN  14.0   BLD 35      76
Name: count, dtype: int64

As we can see: 
- Borough and TDS and location is the exact same;
- There aren't null values for the other dimension.

We will classify this unknown funding source as 'other'. It will not harm the analysis doing so, since the situation is restricted to the same development.

## Meter AMR investigation

In [14]:
dataset['Meter AMR'].value_counts()

Meter AMR
AMR     30000
NONE     4575
Name: count, dtype: int64

It looks like we do not have any information about Interval Meters. This feature will be therefore turned into a boolean one, indicating whether the meter is AMR or not.

## Meter number + location investigation and treatment

- **Assumption**: <u>every different meters serve a single building</u>, while every different building could be served by multiple meters

The **task** here is to check if this assumptions holds. Please read [this](https://www.nyc.gov/site/dep/pay-my-bills/water-meter-faqs.page).

In [15]:
unique_meters_series = dataset.loc[:, 'Meter Number'].value_counts()

unique_meters_series

Meter Number
UN-METERED    170
E20453242      87
V83029959      87
E19354370      87
I13011093      86
             ... 
E15139971       1
O78610245       1
O78106269       1
R22742332       1
R32896781       1
Name: count, Length: 738, dtype: int64

As we can see there are 170 instances with an unknown meter number.

In [16]:
dataset[dataset['Meter Number'] == 'UN-METERED'][
    ['TDS #', 'Location', 'Borough', 'Meter Number']
].value_counts()

TDS #  Location  Borough    Meter Number
343.0  BLD 01    MANHATTAN  UN-METERED      66
2.0    BLD 08    BROOKLYN   UN-METERED      61
       BLD 01    BROOKLYN   UN-METERED      29
37.0   BLD 01    MANHATTAN  UN-METERED      13
377.0  BLD 02    MANHATTAN  UN-METERED       1
Name: count, dtype: int64

First of all we need to treat null locations. We would like to:
1. Count how many (likely) **different** null locations there are, using meter number + TDS;
2. Compare these counts with the number of instances associated to the **same meter number**, found at step 1), on the **whole dataset**. This is useful to make sure that these meters aren't associated with other buildings globally;

In [17]:
# The DataFrame will count the number of meters for every TDS associated with null locations
meter_number_null_location_count_df = dataset[
    dataset['Location'].isna() # consider null locations
][['Meter Number', 'TDS #']].value_counts().reset_index()

# Number of instances in the whole dataset for every meter whose meter_number 
# is equal to the one associated with null locations
temp = dataset[
    dataset['Meter Number'].isin(meter_number_null_location_count_df['Meter Number'])
]['Meter Number'].value_counts()

# Compare the count made considering null locations with the one made considering the whole dataset 
print((meter_number_null_location_count_df['count'].to_numpy() == temp.to_numpy()).all())
meter_number_null_location_count_df

True


,Meter Number,TDS #,count
0,N51387763,51.0,79
1,R32599613,51.0,50
2,N31677722,51.0,50
3,N51387759,51.0,50
4,N29849151,212.0,3
5,N29737078,212.0,3
6,N31034969,274.0,3
7,N30001947,212.0,3
8,N32038857,260.0,3
9,N31451478,226.0,3


- **Assumpion**: since there is no other way to be sure about how many different locations are marked with a NaN, we will assume that in this cases a single building instance is associated with a single meter.

We can define the location mapping like "location_i" where "i" it's a progressive integer that changes every time the location changes, that is every time the meter number change.

In [18]:
# Define location mapping
location_map = {
    v: f'loc_{k}' for k, v in meter_number_null_location_count_df['Meter Number'].to_dict().items()
}

# Fill locations
dataset.loc[dataset['Location'].isna(), 'Location'] = dataset \
                                                        .loc[dataset['Location'].isna(), 'Meter Number'] \
                                                        .map(location_map)

# Fill TDS #
dataset['TDS #'] = dataset['TDS #'].fillna(999)

The following dataframe will basically count the number of instances associated at the triplet (Meter Number, TDS #, Location). To make the assumption true on the whole dataset we want to check if those triplets appear only one time.

In [19]:
meter_number_building_count_df = dataset \
                                    .groupby(by=['Meter Number', 'TDS #', 'Location']) \
                                        .agg(count=('Account Name', len)) \
                                        .sort_values(['TDS #', 'Location']) \
                                        .reset_index()
                    
meter_number_building_count_df.head()

,Meter Number,TDS #,Location,count
0,E20459024,1.0,BLD02 - STORE 6 & 7,1
1,UN-METERED,2.0,BLD 01,29
2,V52268306,2.0,BLD 01,30
3,E19532293,2.0,BLD 08,2
4,UN-METERED,2.0,BLD 08,61


Here we extract a list of the meter associated to multiple buildings.

In [20]:
# multiple meter location
mml_lst = meter_number_building_count_df['Meter Number'] \
            .value_counts()[meter_number_building_count_df['Meter Number'].value_counts() > 1] \
            .index \
            .difference(['UN-METERED']) \
            .to_list()

print(mml_lst)

meter_number_building_count_df[
    meter_number_building_count_df['Meter Number'].isin(mml_lst) 
]

['E20332250', 'G17004105', 'K12499916', 'K15834341', 'K15838320', 'N30898577']


,Meter Number,TDS #,Location,count
32,G17004105,8.0,BLD 02,22
53,G17004105,8.0,BLD 2,1
155,E20332250,209.0,104-33 203RD STREET,1
157,E20332250,209.0,110-26 216TH STREET,84
185,N30898577,209.0,177-48 BAISLEY BOULEVARD,17
579,N30898577,354.0,BLD 04,1
731,K15834341,524.0,BLD 07,13
732,K15834341,524.0,BLD 7,1
733,K12499916,525.0,BLD 08,13
735,K15838320,525.0,BLD 1,1


In [21]:
# Let's consider meter numbers that looks like they are deployed on different buildings
meters = ['E20332250', 'N30898577']

columns = ['TDS #', 'Location', 'Meter Number', 'Development Name', 'Borough']

# Filtered dataframe to inspect via Data Wrangler tool
meter_filtered_df = dataset[
    dataset['Meter Number'].isin(meters)
][columns]

meter_filtered_df

,TDS #,Location,Meter Number,Development Name,Borough
4785,354.0,BLD 04,N30898577,TAPSCOTT STREET REHAB,FHA
4786,209.0,177-48 BAISLEY BOULEVARD,N30898577,FHA REPOSSESSED HOUSES (GROUP I),FHA
4787,209.0,177-48 BAISLEY BOULEVARD,N30898577,FHA REPOSSESSED HOUSES (GROUP I),FHA
4788,209.0,177-48 BAISLEY BOULEVARD,N30898577,FHA REPOSSESSED HOUSES (GROUP I),FHA
4789,209.0,177-48 BAISLEY BOULEVARD,N30898577,FHA REPOSSESSED HOUSES (GROUP I),FHA
...,...,...,...,...,...
32757,209.0,110-26 216TH STREET,E20332250,FHA REPOSSESSED HOUSES (GROUP I),FHA
32758,209.0,110-26 216TH STREET,E20332250,FHA REPOSSESSED HOUSES (GROUP I),FHA
33765,209.0,110-26 216TH STREET,E20332250,FHA REPOSSESSED HOUSES (GROUP I),FHA
34079,209.0,110-26 216TH STREET,E20332250,FHA REPOSSESSED HOUSES (GROUP I),FHA


Most of the duplicates meters are just errors in the location. There are only two cases where it looks like that the same meter is deployed on two different buildings. Let's consider the triplet *(meter number, tds, location)*:

- **(E20332250, 209.0, 104-33 203RD STREET)**: by looking at the *Data Wrangler tool* we can see that the building (209.0, 104-33 203RD STREET) is served by meter *E20257499* since there are 86/87 instances of this building that supports this thesis. We can assume that the actual meter number of this triplet is not *E20332250*, which already serves the building (209.0, 110-26 216TH STREET), but *E20257499*. Since we are not sure we can delete this row.
- **(N30898577, 354.0, BLD 04)**: this looks like really strange. Further analysis are required.

As a final check we can perform the same operation, previously made for meter number, to check if there are other situations where a building (TDS #, location) is under the administration of different boroughs. This situation looks like an anomaly.

In [22]:
columns = ['Borough', 'TDS #', 'Location', 'Account Name']

dataset \
    .groupby(by=['TDS #', 'Location', 'Borough']) \
            .agg(count=('Account Name', len)) \
            .sort_values('TDS #') \
            .reset_index()[['TDS #', 'Location']] \
            .value_counts()

TDS #  Location                 
354.0  BLD 04                       2
2.0    BLD 09                       1
       BLD 13                       1
       BLD 21 - Community Center    1
       BLD16 - STORE 35-36          1
                                   ..
531.0  BLD 06                       1
547.0  BLD 01                       1
       BLD 02                       1
559.0  BLD 01                       1
2.0    BLD 01                       1
Name: count, Length: 526, dtype: int64

As we can see only the couple (354, BLD 04) shows this anomaly. A quick look using the data wrangler tool shows that 17 instances of this couple are located in Brooklyn and only one is actually under FHA. This is our anomaly that we need to fix. 
- The row needs to be **removed**.

## Rate class investigation

In [23]:
dataset['Rate Class'].value_counts()

Rate Class
Basic Water and Sewer         34521
HOT OR COLD WATER IN STORE       90
COMMERCIAL                        9
WATER-METER                       3
MULTIFAMILY                       1
Name: count, dtype: int64

In [24]:
dataset[dataset['Rate Class'].isna()]

,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,...,Service End Date,# days,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (HCF),Water&Sewer Charges,Other Charges
19977,FRANKLIN AVENUE III CONVENTIONAL,BRONX,FRANKLIN AVENUE III CONVENTIONAL,BLD 7,AMR,NaN,524.0,480,B052400,FEDERAL,...,04/03/2014,11,K15834341,N,"1,177.57",NaN,Yes,127,"1,177.57",0
33179,FHA REPOSSESSED HOUSES (GROUP X),FHA,FHA REPOSSESSED HOUSES (GROUP X),loc_14,NaN,NaN,284.0,521,Q028400,FEDERAL,...,04/02/2020,90,K96779329,Y,744.06,NaN,Exception,72,744.06,0
33872,FHA REPOSSESSED HOUSES (GROUP X),FHA,FHA REPOSSESSED HOUSES (GROUP X),loc_14,NaN,NaN,284.0,521,Q028400,FEDERAL,...,01/03/2020,80,K96779329,Y,661.38,NaN,Yes,64,661.38,0


## What about outliers?

Those will be analyzed directly in the pre-processing notebook. The scope of this notebook was to: 
1. Identify why some columns are null;
2. Identify strange values in some attributes.

## Pre-processing strategy

Read the notes on dict.xlsx.